In [ ]:
from ilqr import iLQR
import gym
import numpy as np
import pandas as pd
import time

from aprl.agents import MujocoFiniteDiffDynamicsBasic, MujocoFiniteDiffDynamicsPerformance, MujocoFiniteDiffCost

In [2]:
# Environment setup
env = gym.make('Reacher-v2').unwrapped
env.frame_skip = 1
env.seed(42)
_obs = env.reset()

In [3]:
# Planning setup
N = 100  # planning horizon
us_init = np.array([env.action_space.sample() for _ in range(N)])

dynamics = {
    # Uses env.step(u) directly and finite difference on qpos and qvel directly.
    'my_basic': MujocoFiniteDiffDynamicsBasic(env),
    # Sets ctrl to u directly then uses MuJoCo's forwardSkip to compute qacc.
    # Computes finite difference on qacc, then estimates derivative of qpos and qvel.
    'my_performance': MujocoFiniteDiffDynamicsPerformance(env),
}
x0s = {k: dyn.get_state() for k, dyn in dynamics.items()}

# Finite difference cost

In [ ]:
finite_cost = MujocoFiniteDiffCost(env)
finite_ilqr = iLQR(dynamics, finite_cost, N)

In [ ]:
finite_xs, finite_us = finite_ilqr.fit(x0, us_init, n_iterations=100, on_iteration=on_iteration)

# Analytic cost

In [4]:
import theano
from theano import tensor as T
from ilqr.cost import AutoDiffCost

# Reacher, Gym observation:
# obs[0:1]: xs; np.cos(qpos[0:2]) (qpos[0] is joint0, qpos[1] is joint1)
# obs[2:3]: ys; np.sin(qpos[0:2]);
# obs[4:5]: goal x and y; qpos[2:]; (target_x and target_y)
# obs[6:7]: theta dot
# obs[8:9]: xy of fingertip - target

def make_reacher_cost(kind, control_weight=1.0):
    # qpos[0:3]: theta of joint 0, theta of joint 1; target x and y.
    qpos_inputs = [T.dscalar('theta'), T.dscalar('phi'), T.dscalar('targetx'), T.dscalar('targety')]
    # qvel: derivatives of the above; note target x and y are constant so have derivative zero.
    qvel_inputs = [T.dscalar('thetadot'), T.dscalar('phidot'), T.dscalar('_zero1'), T.dscalar('_zero2')]
    # qacc: second derivatives of qpos. We don't actually use these in the cost.
    qacc_inputs = [T.dscalar('_acc{}'.format(i)) for i in range(len(qpos_inputs))]
    # qacc_warmstart: same shape as qacc
    qacc_warmstart_inputs = [T.dscalar('_accwarm{}'.format(i)) for i in range(len(qpos_inputs))]
    # qfrc_applied: same shape as qacc
    qfrc_applied_inputs = [T.dscalar('_qfrc_applied{}'.format(i)) for i in range(len(qpos_inputs))]
    # xfrc_applied: (5,6)
    xfrc_applied_inputs = [T.dscalar('_xfrc_applied{}'.format(i)) for i in range(5 * 6)]
    if kind == 'mujoco_py':
        # Reacher, MJSimState.flatten():
        # obs[0]: time step, obs[1:4]: qpos[0:3]; obs[5:8]: qvel[0:3]
        # In general might include action and udd_state, but not for Reacher.
        x_inputs = [T.dscalar('_time')] + qpos_inputs + qvel_inputs
    elif kind == 'my_all':
        # Reacher, MujocoRelevantState.flatten()
        x_inputs = qpos_inputs + qvel_inputs + qacc_inputs + qacc_warmstart_inputs + qfrc_applied_inputs + xfrc_applied_inputs
    elif kind == 'my_recommended':
        x_inputs = qpos_inputs + qvel_inputs + qacc_inputs + qacc_warmstart_inputs
    elif kind == 'my_basic_plus':
        x_inputs = qpos_inputs + qvel_inputs + qacc_inputs
    elif kind in ['my_basic', 'my_warmstart', 'my_performance']:
        x_inputs = qpos_inputs + qvel_inputs
    else:
        raise ValueError("Unrecognised kind: '{}'".format(kind))
    u_inputs = [T.dscalar('thetadotdot'), T.dscalar('phidotdot')]
    qpos = T.stack(qpos_inputs)
    u = T.stack(u_inputs)
    
    control_cost = T.dot(u, u)
    target_xpos = qpos[2:4]
    body1_xpos = 0.1 * T.stack([T.cos(qpos[0]), T.sin(qpos[0])])
    fingertip_xpos_delta = 0.11 * T.stack([T.cos(qpos[1]), T.sin(qpos[1])])
    fingertip_xpos = body1_xpos + fingertip_xpos_delta
    delta = fingertip_xpos - target_xpos
    state_cost = T.sqrt(T.dot(delta, delta))
    l = state_cost + control_weight * control_cost
    l_terminal = T.zeros(())
    return AutoDiffCost(l, l_terminal, x_inputs, u_inputs)

In [6]:
def on_iteration(iteration_count, xs, us, J_opt, accepted, converged):
    info = "converged" if converged else ("accepted" if accepted else "failed")
    print("iteration", iteration_count, info, J_opt, xs[-1])

costs = {k: make_reacher_cost(k) for k in dynamics.keys()}
ilqrs = {k: iLQR(dyn, costs[k], N) for k, dyn in dynamics.items()}
xs = {}
us = {}
print(ilqrs.keys())
for k, ilqr in ilqrs.items():
    start = time.time()
    print('*** Fitting {} ***'.format(k))
    x0 = x0s[k]
    xs[k], us[k] = ilqr.fit(x0, us_init, n_iterations=100, on_iteration=on_iteration)
    end = time.time()
    print('*** Fitted {} in {}s ***'.format(k, end - start))

dict_keys(['my_basic', 'my_performance'])
*** Fitting my_basic ***
iteration -1 converged 77.0672963001787 [ 2.32864069 -1.19993367  0.02243766  0.07369059  3.22880925 -4.04084829
  0.          0.        ]
iteration 0 accepted 16.85489457400974 [ 2.6949496  -0.24070435  0.02243766  0.07369059  3.40360537 -3.92220608
  0.          0.        ]
iteration 1 accepted 13.977863492838418 [ 2.84173297  0.20747149  0.02243766  0.07369059  3.52506559 -3.71466247
  0.          0.        ]
iteration 2 accepted 12.170881768875319 [ 2.85318372 -0.05022272  0.02243766  0.07369059  3.6027983  -3.36014486
  0.          0.        ]
iteration 3 failed 12.170881768875327 [ 2.85318372 -0.05022272  0.02243766  0.07369059  3.6027983  -3.36014486
  0.          0.        ]
iteration 4 accepted 12.129830429913762 [ 2.75884007  0.05266829  0.02243766  0.07369059  3.51072004 -3.25887641
  0.          0.        ]
iteration 5 accepted 12.124607048985949 [ 2.56258972  0.37131624  0.02243766  0.07369059  3.39298188 -

iteration 44 accepted 7.553103912646529 [-0.15547401  2.27696798  0.02243766  0.07369059  1.6043329  -1.70183766
  0.          0.        ]
iteration 45 failed 7.553103912646528 [-0.15547401  2.27696798  0.02243766  0.07369059  1.6043329  -1.70183766
  0.          0.        ]
iteration 46 failed 7.553103912646528 [-0.15547401  2.27696798  0.02243766  0.07369059  1.6043329  -1.70183766
  0.          0.        ]
iteration 47 accepted 7.552928529182998 [-0.15664361  2.28831727  0.02243766  0.07369059  1.60988271 -1.68860811
  0.          0.        ]
iteration 48 failed 7.552928529183002 [-0.15664361  2.28831727  0.02243766  0.07369059  1.60988271 -1.68860811
  0.          0.        ]
iteration 49 failed 7.552928529183002 [-0.15664361  2.28831727  0.02243766  0.07369059  1.60988271 -1.68860811
  0.          0.        ]
iteration 50 accepted 7.551963435468378 [-0.1568755   2.29686283  0.02243766  0.07369059  1.6168613  -1.68131666
  0.          0.        ]
iteration 51 failed 7.5519634354683

In [ ]:
res = {k: dyn.f_u(x0s[k], us_init[0], 0) for k, dyn in dynamics.items()}

In [ ]:
res['my_performance']

In [ ]:
res['my_basic']

# Receding horizon

In [ ]:
from ilqr.controller import RecedingHorizonController

def receding(underlying):
    k = 'receding_' + underlying
    dynamics[k] = dynamics[underlying]
    x0s[k] = x0s[underlying]
    controller = RecedingHorizonController(x0s[k], ilqrs[underlying])
    rew = []
    xs[k] = []
    us[k] = []
    for x, u in controller.control(us_init, subsequent_n_iterations=10):
        ob, r, done, info = env.step(u)
        xs[k].append(x)
        us[k].append(u)
        rew.append(r)
        print('iteration', len(rew), r, x, u)
        if len(rew) == N:
            break

In [ ]:
receding('my_basic')
receding('my_performance')

# Rollouts

In [7]:
import time

def rollout(env, dynamics, x0, us, render=False):
    dynamics.set_state(x0)
    if render:
        env.render()
    rew = []
    actual_xs = []
    for u in us:
        _obs, r, done, info = env.step(u)
        rew.append(r)
        actual_xs.append(dynamics.get_state())
        assert not done
        if render:
            env.render()
            time.sleep(0.02)
    return rew, actual_xs

In [8]:
rews = {}
actual_xs = {}
for k, solved_us in us.items():
    print(k)
    rews[k], actual_xs[k] = rollout(env.unwrapped, dynamics[k], x0s[k], solved_us, render=False)
rewards = {k: sum(r) for k, r in rews.items()}
lengths = {k: len(r) for k, r in rews.items()}
pd.DataFrame({'rewards': rewards, 'lengths': lengths})

my_basic
my_performance


,rewards,lengths
my_basic,-12.291561,100
my_performance,-7.492515,100
